In [2]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 80))

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp

def _IMPORT_(x):
    try:
        segs = x.split(' ')
        g = globals()
        if 'github.com' in segs[1]:
            uri = segs[1].replace('github.com', 'raw.githubusercontent.com')
            mod = uri.split('/')
            for s in ['main', 'master']:
                uri = 'https://' + '/'.join(mod[:-1]) + '/main/' + mod[-1] + '.py'
                x = requests.get(uri).text
                if x.status == 200:
                    break
        elif 'gitee.com' in segs[1]:
            mod = segs[1].split('/')
            for s in ['/raw/main/', '/raw/master/']:
                uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:]) + '.py'
                x = requests.get(uri).text
                if x.status == 200:
                    break
        elif segs[1][0] == '/':
            with open(segs[1] + '.py') as fr:
                x = fr.read()
        exec(x, g)
    except:
        pass

def print_progress_bar(x):
    print('\r', end='')
    print('Progress: {}%:'.format(x), '%s%s' % ('▋'*(x//2), '.'*((100-x)//2)), end='')
    sys.stdout.flush()


CPython 3.6.9
IPython 7.16.1

numpy 1.19.4
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed


## Setup Kubernetes 

### 关闭防火墙/Selinux/清空iptables规则/禁用Swap交换分区

In [4]:
%%writefile ./scripts/prepare_environment.sh

sudo ufw disable
sudo ufw status
sudo iptables -P FORWARD ACCEPT

if [[ -f /etc/selinux/config ]]
then
    sudo setenforce 0
    sudo sed -i 's/SELINUX=enforcing/SELINUX=disable' /etc/selinux/config
fi

sudo swapoff -a
sudo sed -i 's/.*swap.*/#&/' /etc/fstab

cat > /etc/sysctl.d/99-kubernetes-cri.conf <<EOF
net.bridge.bridge-nf-call-iptables  = 1
enet.ipv4.ip_forward                = 1
net.bridge.bridge-nf-call-ip6tables = 1
EOF

sysctl --system

modprobe br_netfilter
lsmod | grep "br_netfilter"

Writing ./scripts/prepare_environment.sh


### 安装Docker

In [5]:
%%writefile ./scripts/install_docker.sh

#!/bin/bash

cmd_check=`which docker`
if [[ x$cmd_check == x ]]
then
    sudo apt-get update
    sudo apt-get install apt-transport-https ca-certificates curl software-properties-common -y

    curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -

    sudo add-apt-repository \
        "deb [arch=amd64] https://download.docker.com/linux/ubuntu \
        $(lsb_release -cs) \
        stable"

    sudo apt-get update
    sudo apt-get install docker-ce -y

    cat > /etc/docker/daemon.json << EOF
    {
        "exec-opts": ["native.cgroupdriver=systemd"],
        "log-driver": "json-file",
        "log-opts": {
            "max-size": "100m"
        },
        "storage-driver": "overlay2",
        "runtimes": {
            "nvidia": {
                "path": "nvidia-container-runtime",
                "runtimeArgs": []
            }
        },
        "registry-mirrors": ["https://b9pmyelo.mirror.aliyuncs.com"]
        "default-runtime": "nvidia",
        "insecure-registries": [
            "10.255.0.58:9500"
        ]
    }
    EOF

    mkdir -p /etc/systemd/system/docker.service.d

    systemctl daemon-reload
    systemctl restart docker
fi

# sudo usermod -aG docker $USER
# newgrp docker

docker version

Overwriting ./scripts/install_docker.sh


### [安装kubeadm,kubelet,kubectl](https://www.downloadkubernetes.com/)

国内下载: https://mirrors.ustc.edu.cn/kubernetes/apt/pool/

方法一:

In [6]:
%%writefile ./scripts/install_k8s.sh
#!/bin/bash

if [[ x$(which kubectl) == x || x$(which kubelet) == x ]]
then
    RELEASE=v1.22.0 # "$(curl -sSL https://dl.k8s.io/release/stable.txt)"
    cd /usr/bin
    sudo curl -L --remote-name-all https://dl.k8s.io/${RELEASE}/bin/linux/amd64/{kubeadm,kubelet,kubectl}
    sudo chmod +x {kubeadm,kubelet,kubectl}
    sudo apt install bash-completion
    sudo kubectl completion bash > /etc/bash_completion.d/kubectl
fi
kubeadm version
kubelet version
kubectl version --client

Overwriting ./scripts/install_k8s.sh


方法二:

In [7]:
%%writefile ./scripts/install_k3s.sh
#!/bin/bash
SRC_GPG=https://gitee.com/qrsforever/blog_source_assets/raw/master/Tutorial/k8s/apt-key.gpg
DST_GPG=/usr/share/keyrings/kubernetes-archive-keyring.gpg
sudo curl -XGET -L $SRC_GPG -o $DST_GPG
echo "deb [signed-by=$DST_GPG] http://mirrors.ustc.edu.cn/kubernetes/apt kubernetes-xenial main" | sudo tee /etc/apt/sources.list.d/kubernetes.list
sudo apt update
sudo apt install -y kubeadm kubelet kubectl kubernetes-cni 
kubeadm version && kubelet --version && kubectl version

Overwriting ./scripts/install_k3s.sh


Resume when reboot:

```sh
sudo systemctl start kubelet
sudo systemctl enable kubelet
```

### Pull Images

In [8]:
%%writefile ./scripts/aliyuncs_image_pull.sh
#!/bin/bash

for image_uri in $(kubeadm config images list)
do
    echo "pull $image_uri ..."
    exist=`docker image ls $image_uri  --format "{{.Tag}}"`
    if [[ x$exist == x ]]
    then
        image_name=`echo $image_uri | cut -d\/ -f2-`
        # image_name=`echo $image_uri | rev | cut -d\/ -f1 | rev`
        name=${image_name%%:*}
        if [[ ${name} == "coredns/coredns" ]]
        then
            version=${image_name##*:v}
            docker pull registry.cn-hangzhou.aliyuncs.com/google_containers/coredns:${version}
            docker tag registry.cn-hangzhou.aliyuncs.com/google_containers/coredns:${version} k8s.gcr.io/${image_name}
            docker rmi registry.cn-hangzhou.aliyuncs.com/google_containers/coredns:${version}
        else
            docker pull registry.cn-hangzhou.aliyuncs.com/google_containers/$image_name
            docker tag registry.cn-hangzhou.aliyuncs.com/google_containers/$image_name k8s.gcr.io/$image_name
            docker rmi registry.cn-hangzhou.aliyuncs.com/google_containers/$image_name
        fi
    fi
done

Overwriting ./scripts/aliyuncs_image_pull.sh


### Init Cluster

```bash
sudo kubeadm init --pod-network-cidr=10.255.0.0/16 --apiserver-advertise-address=10.255.0.103
```

### Work Node Join

```bash
# Get token
kubeadm token list
# Get sha256
openssl x509 -pubkey -in /etc/kubernetes/pki/ca.crt | openssl rsa -pubin -outform der 2>/dev/null | openssl dgst -sha256 -hex | sed 's/^.* //'
```

```sh
# Join cluster
TOKEN=i7s1ld.ebnzbseprzlminbe
SHA256=3f70106afa50318ba3a9d65810f04768661e63a3e4d44582559793d6681f312b
sudo kubeadm join 10.255.0.103:6443 --token $TOKEN --discovery-token-ca-cert-hash sha256:$SHA256
```

### Deploy Flannel Network

```sh
sudo kubectl apply -f https://raw.githubusercontent.com/coreos/flannel/master/Documentation/kube-flannel.yml
```

### One Application on Kubernetes

In [ ]:
```sh
sudo kubectl create deployment nginx --image=nginx
sudo kubectl get deployments nginx
sudo kubectl create service nodeport nginx --tcp=80:80
sudo kubectl get svc
sudo kubectl get pods --all-namespaces -o wide
```

## [安装kfctl](https://github.com/kubeflow/kfctl)

In [9]:
KFCTL_TARGZ = 'https://github.com//kubeflow/kfctl/releases/download/v1.2.0/kfctl_v1.2.0-0-gbc038f9_linux.tar.gz'

In [10]:
%%template_write_script ./scripts/install_kfctl.sh
#!/bin/bash

cmd_check=`which kfctl`
if [[ x$cmd_check == x ]]
then
    curl -X GET https://github.91chifun.workers.dev/{KFCTL_TARGZ} -o /tmp/kfctl.tar.gz
    sudo tar -xvf /tmp/kfctl.tar.gz -C /usr/bin
fi
kfctl version

## [安装kustomize](https://github.com/kubernetes-sigs/kustomize)

In [11]:
KUSTOMIZE_TARGZ = 'https://github.com/kubernetes-sigs/kustomize/releases/download/kustomize%2Fv4.2.0/kustomize_v4.2.0_linux_amd64.tar.gz'

In [12]:
%%template_write_script ./scripts/install_kustomize.sh
#!/bin/bash
cmd_check=`which kustomize`
if [[ x$cmd_check == x ]]
then
    curl -XGET https://github.91chifun.workers.dev/{KUSTOMIZE_TARGZ} -o /tmp/kustomize.tar.gz
    tar -xvf /tmp/kustomize.tar.gz -C /usr/bin
fi
kustomize version

## [下载manifests](https://github.com/kubeflow/manifests)

## References

- https://adamtheautomator.com/install-kubernetes-ubuntu/
- https://towardsdatascience.com/kubeflow-pipelines-how-to-build-your-first-kubeflow-pipeline-from-scratch-2424227f7e5
- https://medium.com/clarusway/kubernetes-step-by-step-setup-guide-for-beginners-cba307250a6c
- https://blog.csdn.net/apl359/article/details/119123177
- https://zhuanlan.zhihu.com/p/46341911
- https://medium.com/jacky-life/%E4%BD%BF%E7%94%A8-kubeadm-%E5%AE%89%E8%A3%9D-k8s-abe1631aa600
- https://kubernetes.io/zh/docs/setup/production-environment/tools/kubeadm/install-kubeadm/
- https://blog.csdn.net/kozazyh/article/details/88232705

## FAQs

- [kubectl安装无法连接packages.cloud.google.com][1]
- [the server localhost:8080 was refused][2]
- [kubelet isn't running or healthy][3]
- [部署Kubernetes(k8s)时，为什么要关闭swap、selinux、防火墙？][4]
- [部署前准备(swap、selinux等)][5]
- [k8s查看token,discovery-token-ca-cert-hash][6]
- [ERROR IsPrivilegedUser][7]
- [How to gracefully remove a node from Kubernetes?][8]
- [Pod容器内部无法ping 通ClusterIP][9]

  
[1]: https://blog.csdn.net/qq_39698985/article/details/114983293
[2]: https://blog.csdn.net/leenhem/article/details/119736586
[3]: https://www.jianshu.com/p/f53650a85131
[4]: https://www.zhihu.com/question/374752553
[5]: https://www.programminghunter.com/article/9956368399/
[6]: https://blog.csdn.net/weixin_43968936/article/details/103688562
[7]: https://blog.csdn.net/weixin_30872337/article/details/97591467
[8]: https://stackoverflow.com/questions/35757620/how-to-gracefully-remove-a-node-from-kubernetes
[9]: https://blog.csdn.net/weixin_39950057/article/details/111537049?utm_medium=distribute.pc_relevant.none-task-blog-2~default~baidujs_title~default-0.base&spm=1001.2101.3001.4242